# Loading most recent data from postgres table

## Installing needed packages and connections

In [1]:
# Install libraries
!pip install psycopg2-binary
!pip install requests
!pip install sqlalchemy --upgrade
!pip install dask --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 28.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.0/157.0 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 27.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.7/613.7 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: dask
    Found existing installation: dask 2023.2.0
    Uninstalling dask-2023.2.0:
      Successfully uninstalled dask-2023.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2023.2.0 requires dask==2023.2.0, but you have dask 2023.4.1 which is i

## Load data and analytics using Dask

In [3]:
import os
import dask.dataframe as dd
from api_query import api_query_call

df_pandas = api_query_call("SELECT * FROM uniswap_table")


In [4]:
df = dd.from_pandas(df_pandas, npartitions=10)
df.head()

,tradeindex,block_timestamp_time,block_height,exchange_fullname,timeinterval_minute,trades,buyamount,sellamount,tradeamount,basecurrency_symbol,quotecurrency_symbol,transaction_hash,transaction_gas,transaction_to_address,transaction_txfrom_address
0,6,2023-05-09 01:30:59,17219643,Uniswap,2023-05-09 01:30:00,1,112.27823506391793,126.45409443151638,112.27823506391793,ETH,REN,0x72760a699692e40e04a2b19b402a23cfde597a10d655...,300328,0x0eae044f00b0af300500f090ea00027097d03000,0xb543f8d66765c7074d1d77147d7e07880c96908f
1,6,2023-05-09 01:30:59,17219643,Uniswap,2023-05-09 01:30:00,1,112.27823506391793,126.45409443151638,112.27823506391793,REN,ETH,0x72760a699692e40e04a2b19b402a23cfde597a10d655...,300328,0x0eae044f00b0af300500f090ea00027097d03000,0xb543f8d66765c7074d1d77147d7e07880c96908f
2,22,2023-05-09 00:36:47,17219375,Uniswap,2023-05-09 00:36:00,1,0.0,252.77523224204333,252.77523224204333,ETH,BLT,0x5d60174afa5297406dcf76b78e2c3d901592a9aefa1d...,707931,0x0ef8b4525c69bfa7bdece3ab09f95bbf0944b783,0x25173f370af28592354098a18e583f8eaa7ab264
3,22,2023-05-09 00:36:47,17219375,Uniswap,2023-05-09 00:36:00,1,0.0,252.77523224204333,252.77523224204333,BLT,ETH,0x5d60174afa5297406dcf76b78e2c3d901592a9aefa1d...,707931,0x0ef8b4525c69bfa7bdece3ab09f95bbf0944b783,0x25173f370af28592354098a18e583f8eaa7ab264
4,6,2023-05-09 00:13:23,17219260,Uniswap,2023-05-09 00:13:00,1,326.93109400760903,283.5812411661728,283.5812411661728,GEN,ETH,0x3e8c14e18522b5d7282b27524a6c94a8806f26ab19fa...,250465,0x0b8a49d816cc709b6eadb09498030ae3416b66dc,0x2eb393fbac8aaa16047d4242033a25486e14f345


In [6]:
#Statistics
sum_stats = df.describe().compute()
print(sum_stats)

        tradeindex  block_height  trades  transaction_gas
count  1002.000000  1.002000e+03  1002.0     1.002000e+03
mean      6.231537  1.714689e+07     1.0     2.774886e+05
std       5.315828  3.864360e+04     0.0     2.459517e+05
min       2.000000  1.708408e+07     1.0     5.184700e+04
25%       3.000000  1.711956e+07     1.0     1.898490e+05
50%       5.000000  1.713896e+07     1.0     2.476660e+05
75%      11.000000  1.718058e+07     1.0     4.131120e+05
max      47.000000  1.721964e+07     1.0     2.036563e+06


In [7]:
print('Most common "from" wallet addresses:')
df['transaction_txfrom_address'].value_counts().head()

Most common "from" wallet addresses:


0xd7e1236c08731c3632519dcd1a581bfe6876a3b2    144
0xb543f8d66765c7074d1d77147d7e07880c96908f     64
0xa06c3c08a19e51b33309eddfb356c33ead8517a3     34
0x2ee36e41387f87b7e6f678a86d1e575b23b996f5     34
0xbda27a23fb386024c495ad2f6b834cbfe829b913     20
Name: transaction_txfrom_address, dtype: int64

In [74]:
print('Check that no overlaps of same wallet in to and from category:')
sum(df['transaction_txfrom_address'] == df['transaction_to_address'])

Check that no overlaps of same wallet in to and from category:


0

In [59]:
# top transactions and addresses
counts_address = df['transaction_to_address'].value_counts()

print(counts_address.compute()[:10])

0x24902aa0cf0000a08c0ea0b003b0c0bf600000e0    164
0xdef1c0ded9bec7f1a1670819833240f027b25eff    128
0x0eae044f00b0af300500f090ea00027097d03000     86
0x0000000000007f150bd6f54c40a34d7c3d5e9f56     76
0x0000000000a84d1a9b0063a910315c7ffa9cd248     76
0xe66b31678d6c16e9ebf358268a790b763c133750     70
0x1111111254eeb25477b68fb85ed929f73a960582     46
0x76f4eed9fe41262669d0250b2a97db79712ad855     46
0xe4000004000bd8006e00720000d27d1fa000d43e     42
0x881d40237659c251811cec9c364ef91dc08d300c     36
Name: transaction_to_address, dtype: int64


In [75]:
df_top1 = df[df.transaction_to_address == counts_address.index.compute()[0]]   
df_top1.describe().compute()

,tradeindex,block_height,trades,transaction_gas
count,164.000000,1.640000e+02,164.0,164.000000
mean,6.853659,1.714627e+07,1.0,219741.597561
std,4.461483,3.797255e+04,0.0,121427.369346
min,3.000000,1.708408e+07,1.0,117810.000000
25%,4.250000,1.712058e+07,1.0,141635.000000
50%,6.000000,1.713933e+07,1.0,177759.000000
75%,8.000000,1.717987e+07,1.0,308002.000000
max,25.000000,1.721209e+07,1.0,730533.000000
